## Basic setup and directory structure

In [ ]:
import os

BASE_DIR = '/home/rithomas'
# folder containing MIDI and/or MusicXML files. can have child folders.
RAW_DATA = os.path.join(BASE_DIR, 'data', 'clean_midi')
# TFRecord file that will contain NoteSequence protocol buffers.
SEQUENCES_TFRECORD = os.path.join(BASE_DIR, 'cache', 'melodyRNN', 'data', 'notesequences.tfrecord')
# CONFIG=<one of 'one_drum' or 'drum_kit', matching the bundle>
DRUMS_RNN_CONFIG = 'drum_kit'
# Model directory for storing weights and tensorboard stuff
DRUMS_RNN_DIR = os.path.join(BASE_DIR, 'cache', 'drumsRNN', 'model')
# Dataset of Sequence Examples for the ImprovRNN Model
DRUMS_RNN_DATASET = os.path.join(BASE_DIR, 'cache', 'drumsRNN', 'data')
# Training sequence examples
TRAINING_EXAMPLES = os.path.join(DRUMS_RNN_DATASET, 'training_drum_tracks.tfrecord')
# Dir for testing outputs
OUTPUTS_PATH = os.path.join(DRUMS_RNN_DATASET, 'outputs')
# Final bundled model
BUNDLE_DIR = os.path.join(DRUMS_RNN_DIR, DRUMS_RNN_CONFIG + '.mag')

if not os.path.exists(SEQUENCES_TFRECORD):
    !python3 "magenta/scripts/convert_dir_to_note_sequences.py" \
      --input_dir=$RAW_DATA \
      --output_file=$SEQUENCES_TFRECORD \
      --recursive

## Create DrumsRNN Dataset

In [ ]:
!python3 "magenta/models/drums_rnn/drums_rnn_create_dataset.py" \
    --config=$DRUMS_RNN_CONFIG \
    --input=$SEQUENCES_TFRECORD \
    --output_dir=$DRUMS_RNN_DATASET \
    --eval_ratio=0.10

## Train DrumsRNN on Dataset

In [ ]:
!python3 "magenta/models/drums_rnn/drums_rnn_train.py" \
    --config=$DRUMS_RNN_CONFIG \
    --run_dir=$DRUMS_RNN_DIR \
    --sequence_example_file= $TRAINING_EXAMPLES \
    --hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
    --num_training_steps=20000

## Generate some outputs for testing

In [ ]:
!python3 "magenta/models/drums_rnn/drums_rnn_generate.py" \
    --config=$DRUMS_RNN_CONFIG \
    --run_dir=$DRUMS_RNN_DIR \
    --hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
    --output_dir=$OUTPUTS_PATH \
    --num_outputs=10 \
    --num_steps=128 \
    --primer_drums="[(36,)]"

## Save model if you like it!

In [ ]:
!python3 "magenta/models/drums_rnn/drums_rnn_generate.py" \
    --config=$DRUMS_RNN_CONFIG \
    --run_dir=$DRUMS_RNN_DIR\
    --hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
    --bundle_file=$BUNDLE_DIR \
    --save_generator_bundle